## Data division by train, val, test sets.

In [1]:
import pandas as pd
import numpy as np
import warnings
import torch

torch.manual_seed(420)
np.random.seed(420)
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_table("../data/raw/filtered.tsv")

In [10]:
data = data.drop(columns=["Unnamed: 0"])

In [11]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2):
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

train, val, test= train_validate_test_split(data)

In [12]:
print(train.shape)
train.head()

(346666, 6)


,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
81743,"I need a win, damn it, I need a win.",I need something to win.,0.813266,0.324324,0.979191,0.000067
260873,I don't have time for that kind of bullshit.,I don't have time for this.,0.636570,0.377778,0.999253,0.000049
538204,Just be quiet!,"shut up, Jew!",0.656237,0.066667,0.000941,0.999635
529017,It's crazy.,he's crazy!,0.908744,0.000000,0.000464,0.993526
84580,She stamped her foot to show she knew as well ...,she stamped out to show she knew well like eve...,0.679127,0.091837,0.768190,0.000181


In [13]:
print(val.shape)
val.head()

(115555, 6)


,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
306509,How did you end up here?,how the hell did you end up?,0.895624,0.137931,0.000046,0.928709
260011,"Methods of killing-- automatic rifle, shotgun,...","automatic rifle, shotgun, handgun, handgun, ar...",0.854687,0.315217,0.990555,0.023346
251767,"No, he'd die before he spilled his guts.","no, he would rather die than divulge anything.",0.605975,0.127660,0.980841,0.088504
269508,"That guy's seriously bad news, dude.","this guy is really bad, man.",0.827429,0.216216,0.000374,0.918747
443929,"So tell me, cos I'd like to know, what on eart...","now, please tell me 'cause I'd really like to ...",0.838692,0.132867,0.001679,0.997845


In [14]:
print(test.shape)
test.head()

(115556, 6)


,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
381843,The boys would like to engrave your name here ...,the boys would like you to put a name on the b...,0.726304,0.190476,0.000796,0.994487
572881,"I won't let you kill her, Yagami Light!","I won't let you do that, Yagami Lighte!",0.709166,0.000000,0.982598,0.000248
529131,Pathetically.,woefully.,0.819898,0.285714,0.980882,0.000223
456230,"Well, if you girls play your cards right, you ...","if you take it at the right end, you'll knock ...",0.651357,0.118421,0.000141,0.937102
187675,"Laura, Cindy, this shit is totally gratuitous.","Laura, Cindy, this is completely undeserved.",0.819292,0.042553,0.999280,0.000047


In [17]:
train.to_csv("../data/internal/train.csv")
val.to_csv("../data/internal/validation.csv")
test.to_csv("../data/internal/test.csv")